```
         Copyright Rein Halbersma 2020-2021.
Distributed under the Boost Software License, Version 1.0.
   (See accompanying file LICENSE_1_0.txt or copy at
         http://www.boost.org/LICENSE_1_0.txt)
```

# Dynamic programming for the Frozen Lake
An implementation of the dynamic programming assignment of the [Udacity Deep Reinforcement Learning Nanodegree](https://github.com/udacity/deep-reinforcement-learning/blob/master/dynamic-programming/Dynamic_Programming_Solution.ipynb)

In [1]:
from itertools import product

import gym
import numpy as np

from doctrina.algorithms import dp
from doctrina.spaces import state_table

env = gym.make('FrozenLake-v0', is_slippery=True)

## Part -1: Adapt the Gym environment
> The reasonable man adapts himself to the world: the unreasonable one persists in trying to adapt the world to himself. Therefore all progress depends on the        unreasonable man.”
>
>    ― George Bernard Shaw, Man and Superman

### Expand the state space to include a terminal state
We must be careful to distinguish episodic tasks from continuing tasks. For episodic tasks (such as `FrozenLake-v0`), Sutton & Barto (p.54) recommend introducing a terminal state in which all episodes end. This distinguishes the set of all nonterminal states, denoted $S$, from the set of all states plus the terminal state, denoted $S^{+}$. 

The authors of the Frozen Lake environment did not follow this advice and instead encode the terminal nature of the goal state and the holes in the lake by the `done` flag. For the Frozen Lake environment this still leads to the correct results. But for more general environments this is not necessarily the case because the terminal nature of a state can be action-dependent. 

In the Blackjack environment, a hand will always terminate after standing, but after hitting it depends on whether the player busts or not. For such environments an explicit terminal state is required to assign a unique value to each state. To avoid having to analyze a specific environment's properties beforehand, we always create a terminal state for episodic tasks, even when it is not strictly necessary.

For continuing tasks (such as Jack's Car Rental), no terminal state is required but one does need to set the discount parameter $\gamma \lt 1$.

In [2]:
H = W = int(np.sqrt(env.nS))
env.observation_shape = (H, W)
terminal = env.nS
env.nSp = env.nS + 1

### Extract unique rewards
We need to know the (sorted) vector of unique rewards in order to determine the size of the dense tensor representing `env.P`.

In [3]:
Reward = np.unique([
    t[2]
    for s, a in product(range(env.nS), range(env.nA))
    for t in env.P[s][a]
])
nR = len(Reward)

### Create a dense version of `env.P`
For reasons of both clarity and performance, it's more convenient to change the dictionary of dictionaries of lists `env.P` into a dense tensor that directly models equation (3.2) in Sutton & Barto. This allows the use of NumPy matrix multiplication when implementing the Bellman-equation. The transformation between the two equivalent representations is a relative straightforward exercise that can be applied to any model-based Gym environment found on the internet. The transformation is reminiscent of the transformation between adjacency lists and adjacency matrices for graph representations.

In [4]:
# Equation (3.2) in Sutton & Barto (p.48):
# p(s', r|s, a) = probability of transition to state s' with reward r, from state s and action a.
P_tensor = np.zeros((env.nSp, env.nA, env.nSp, nR))
P_tensor[terminal, :, terminal, 0] = 1
for s, a in product(range(env.nS), range(env.nA)):
    for prob, next, reward, done in env.P[s][a]:
        P_tensor[s, a, terminal if done else next, int(reward)] += prob
# Equation (3.3) in Sutton & Barto (p.48):
assert np.isclose(P_tensor.sum(axis=(2, 3)), 1).all()

### Avoid redundant computations (aka Once And Only Once)
The two terms in the Bellman-equation both contain summations that are state-independent. This redundancy can be avoided by pre-computing a separate transition tensor and a a reward matrix as discussed in Sutton & Barto (p.49). For the reward matrix, we need a (sorted) vector of unique rewards.

In [5]:
# Equation (3.4) in Sutton & Barto (p.49):
# p(s'|s, a) = probability of transition to state s', from state s taking action a.
env.transition = P_tensor.sum(axis=3)

In [6]:
# Equation (3.5) in Sutton & Barto (p.49):
# r(s, a) = expected immediate reward from state s after action a.
env.reward = P_tensor.sum(axis=2) @ Reward

## Part 0: Explore FrozenLakeEnv

In [7]:
# print the state space and action space
print(env.observation_space)
print(env.action_space)

# print the total number of states and actions
print(env.nS)
print(env.nA)

Discrete(16)
Discrete(4)
16
4


In [8]:
env.P[1][0]

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 5, 0.0, True)]

## Part 1: Iterative Policy Evaluation

In [9]:
random_policy = dp.policy_init_stoch(env)
V, delta, iter = dp.V_policy_eval_stoch_sync(env, random_policy)
print(state_table(V[:-1], env).round(5))
print(delta, iter)

[[0.01394 0.01163 0.02095 0.01048]
 [0.01625 0.      0.04075 0.     ]
 [0.03481 0.08817 0.14205 0.     ]
 [0.      0.17582 0.43929 0.     ]]
9.372906270566084e-09 74


## Part 2: Obtain $q_\pi$ from $v_\pi$

In [10]:
Q = dp.Q_from_V_sync(env, V, gamma=1.)
print(Q[:-1])

[[0.01470938 0.01393976 0.01393976 0.01317013]
 [0.00852355 0.0116309  0.01086128 0.01550787]
 [0.02444513 0.02095296 0.02406032 0.01435344]
 [0.01047648 0.01047648 0.00698431 0.01396863]
 [0.02166486 0.01701827 0.01624865 0.0100628 ]
 [0.         0.         0.         0.        ]
 [0.05433537 0.04735105 0.05433537 0.00698432]
 [0.         0.         0.         0.        ]
 [0.01701827 0.04099204 0.03480619 0.04640825]
 [0.07020885 0.1175599  0.10595784 0.05895311]
 [0.18940421 0.17582036 0.16001423 0.04297382]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.08799676 0.20503718 0.23442715 0.17582036]
 [0.25238823 0.53837051 0.52711477 0.43929117]
 [0.         0.         0.         0.        ]]


## Part 3: Policy Improvement

In [11]:
policy = dp.V_policy_impr_stoch(env, V, gamma=1.)
print(policy[:-1])

[[1.   0.   0.   0.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.5  0.   0.5  0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]


## Part 4: Policy Iteration

In [12]:
policy, V, info = dp.V_policy_iter(env, stoch=True)
print(policy[:-1])
print(state_table(V[:-1], env).round(5))
print(info)

[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.5  0.   0.5  0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]
[[0.82353 0.82353 0.82353 0.82353]
 [0.82353 0.      0.52941 0.     ]
 [0.82353 0.82353 0.76471 0.     ]
 [0.      0.88235 0.94118 0.     ]]
{'delta': 9.513950849360242e-09, 'evaluations': 1078, 'improvements': 4}


## Part 5: Truncated Policy Iteration

In [13]:
policy, V, info = dp.V_policy_iter(env, stoch=True, maxiter=2)
print(policy[:-1])
print(state_table(V[:-1], env).round(5))
print(info)

[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.5  0.   0.5  0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]
[[0.82353 0.82353 0.82353 0.82353]
 [0.82353 0.      0.52941 0.     ]
 [0.82353 0.82353 0.76471 0.     ]
 [0.      0.88235 0.94118 0.     ]]
{'delta': 9.935870903809985e-09, 'evaluations': 830, 'improvements': 415}


## Part 6: Value Iteration

In [14]:
policy, V, info = dp.V_value_iter(env, stoch=True)
print(policy[:-1])
print(state_table(V[:-1], env).round(5))
print(info)

[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.5  0.   0.5  0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]
[[0.82353 0.82353 0.82353 0.82353]
 [0.82353 0.      0.52941 0.     ]
 [0.82353 0.82353 0.76471 0.     ]
 [0.      0.88235 0.94118 0.     ]]
{'delta': 9.96687876675395e-09, 'iter': 618}


## Part 7: Purrformance (aka The Need for Speed)

In [15]:
%timeit dp.V_value_iter(env, stoch=True)

9.65 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
